# Exploring [Commercial Vibrancy in Chicago](https://www.jpmorganchase.com/corporate/institute/insight-commercial-vibrancy-of-chicago.htm)

In March of 2017, the [JPMorgan Chase Institute](https://www.jpmorganchase.com/corporate/institute/institute.htm) released a report called [**Going the Distance:** Big Data on Resident Access to Everyday Goods](https://www.jpmorganchase.com/corporate/institute/document/institute-access-to-consumption-brief.pdf). The report leveraged the ability to capture the distance between a consumer's home zip code, and the zip code in which the merchant resided, for millions of transactions. We refer to this distance as the "merchant distance". When merchant distances are collected within a city, we can analyze how they differ in different parts of the city, and for different populations. Such analysis yields a novel view of within-city demand; it also has the power to speak to inequality in access to amenities across subpopulations, offering a new window into the lived experience of millions of people.

The initial report focused on Detroit and New York, but a more recent analysis has centered on Chicago. The analysis performed yielded a [publicly available data file](https://www.jpmorganchase.com/corporate/institute/document/chicago_data_2016q2.zip) which holds the median "merchant distance" by zip code across all observed purchases. It also holds median merchant distances for grocery and restaurant purchases, in addition to overall merchant distances for low- and high-income residents of Chicago.

In this Notebook, we will create a map of this data, to better the commercial landscape in Chicago.

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
import geopandas as gpd
import json

# Visualization
import bokeh.plotting as bp
import bokeh.models as bm
import bokeh.io as bio
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
import IPython.display as ipd #show iframes

# Color mapping is handled well by matplotlib, e.g., normalizers
import matplotlib as mpl
import matplotlib.cm as cmap

# Define function to allow us to display charts in an iframe
def show_iframe(url, iheight=400, iwidth=1000):
    display_string = '<iframe src={url} width={w} height={h}></iframe>'.format(url=url, w=iwidth, h=iheight)
    print(display_string)
    return ipd.HTML(display_string)

## Data Read

The data we need for this exercise comes from two sources. The first is just the "flat" CSV file (referenced above) that contains the access data by zip code in Chicago. This would be all we needed if we were concerned only with making non-spatial visualizations. However, since we are going to make a map, we need some way of associating the zip code level data with actual shapes that represent zip codes in Chicago. To be more precise, we need shapes represented as a set of coordinates, and then we need to associated specific data values with each coordinate set. We have taken the liberty of already downloading the flat file and depositing it in `../data/chicago_data_2016q2.csv`. Note that missing values indicate that we did not see enough customers and transactions in a given zip code to clear our minimum reporting threshold.

In [ ]:
# Read in data
access = pd.read_csv('../data/chicago_data_2016q2.csv')

access.head()

#overall: median distance between home and business ZIP codes

To get the spatial information, we are going to use a [GeoJSON](http://geojson.org/) file (`.geojson`) because they are easier to work with in Python.  (Check out this [great post](https://macwright.org/2015/03/23/geojson-second-bite) about GeoJSON in depth.) Remember that [JSON](http://www.json.org/) files (of which GeoJSON is a special case) are structured in a very similar way to [Python dictionaries](https://docs.python.org/3/tutorial/datastructures.html#dictionaries). Consequently, once we read them in, we have all kinds of great built-in methods for manipulating them. The tool set for working with shapefiles (`.shp`) files is just not as robust. That said, shapefiles are far more ubiquitous than GeoJSON files because [ESRI](https://www.esri.com/en-us/home) has spent years saturating the market with their (initially) proprietary format. This is likely the reason that all of the data that the Census offers in its [Topologically Integrated Geographic Encoding and Referencing](https://www.census.gov/geo/maps-data/data/tiger.html) (TIGER) products are available as shapefiles.  Consequently, we will need to convert our spatial data files from shapefiles to GeoJSON. 

To prepare this example, I have used the [Geospatial Data Abstraction Library](http://gdal.org/) (GDAL) to convert the Census ZCTA5 shapefile to GeoJSON:

    ogr2ogr -f GeoJSON -t_srs crs:84 tl_2017_us_zcta510.geojson tl_2017_us_zcta510.shp
    
That said, you don't have to install GDAL to convert the file you download from the Census. If you like, you can use one of the many available online converters that can accept shapefiles and return GeoJSON. **(Literally just search "convert shapefile to geojson", and Google will know where you are going halfway through "shapefile".)**  Let's take a quick look at this puppy.

In [ ]:
!ls -la ../data/spatial/
!head -5 ../data/spatial/tl_2017_us_zcta510.geojson

In [ ]:
!tail -3 ../data/spatial/tl_2017_us_zcta510.geojson

The first thing to note is the size of the GeoJSON file.  It is a rather beastly 1.4 GB, so we definitely do not want to read this whole thing into memory.  (Here are some [handy tips](https://www.dataquest.io/blog/python-json-tutorial/) about working with large JSON assets.) We need to find some way of reading in only Chicago zip codes. The second call (which prints out the head of the GeoJSON file) gives us clues as to how to achieve this goal.  The general structure of the file is as follows:

    {
        "type": "FeatureCollection",
        "crs": { 
            "type": "name", 
            "properties": { ... } 
            },
        "features": [
            { 
                "type": "Feature", 
                "properties": { ... }, 
                "geometry": { 
                    "type": "Polygon", 
                    "coordinates": [ ... ]
                 }
            }
        ]
    }
    
    
Coordinates: Features->Geometry->Coordinates
Properties: Features->Properties

The vast majority of the information in this structure resides in `f["features"]["geometry"]["coordinates"]`. To make sure we only carry the coordinates we need, we should leverage the filtering opportunity that `f["features"]["properties"]` provides. If we take a closer look at the first feature, we can see all the data values offered for each zip code:

    "properties": { 
        "ZCTA5CE10": "84049", 
        "GEOID10": "84049", 
        "CLASSFP10": "B5", 
        "MTFCC10": "G6350", 
        "FUNCSTAT10": "S", 
        "ALAND10": 215986428, 
        "AWATER10": 10468874, 
        "INTPTLAT10": "+40.5228727", 
        "INTPTLON10": "-111.5243678" 
    }
    
Hopefully `ZCTA5CE0` sticks out as a high-value target, since our flat file already gave us a list of the ZIP codes we can map.

In [ ]:
# Grab the set of zip codes
zips = sorted(set(access['zip_code']))

print(zips)

#use "set" here because "set" returns unique values only

Our goal is to build a list of features that corresponds to only our Chicago zip codes. At a high level, we need to do the following:

1. Create a list of Chicago zip codes
2. Loop over the zip codes, scanning the file for lines that contain each one
3. If a line holds a Chicago zip code, deposit the feature in a list and move to the next zip code. If it doesn't, simply move to the next one.

Yet again, behold the power of IPython's ability to blend bash commands with Python logic and syntax.  We don't want to read in the whole file before filtering, because that will take forever. Furthermore, bash already has an excellent tool for finding substrings of relevance ("IPython" is a substring of "IPython for the win"): [grep](https://www.gnu.org/savannah-checkouts/gnu/grep/manual/grep.html).  To provide an example of the core functionality that will make this work, note how easy and fast it is to find the feature information pertaining to the first Chicago zip code, `60601`.

In [ ]:
first_zip = !grep '"ZCTA5CE10": "60601"' ../data/spatial/tl_2017_us_zcta510.geojson

first_zip

We are now going to use this compact approach to populate features in a JSON string modeled after the original GeoJSON file. Then we can add the GeoJSON front end to our feature list, make sure we cap off the features with closing brackets, and we will be in business.

In [ ]:
# Create dictionary to hold new subset of spatial info
chicago = '''
{
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
    "features": [
'''

# Create container to hold feature-specific substrings
chi_zips = []
# For each Chicago zip code
for z in zips:
    print('Scanning for {}'.format(z))
    # grab the relevant feature info and throw it in the chicago dictionary
    z_tmp = !grep '"ZCTA5CE10": "{z}"' ../data/spatial/tl_2017_us_zcta510.geojson
    chi_zips.append(z_tmp[0][:-1])
    #bash results are collected as a list, and we want to drop trailing commas
    #append Chicago ZIPs with the feature name
    #"[0]" pull out of a list, otherwise it will mess up your JSON
    #"[:-1]"=everything but the last character (remove trailing comma)
    
chicago_json = chicago + (',').join(chi_zips) + ']}'
    
    #use a JSON linter: https://jsonlint.com/

In [ ]:
print(chicago_json)

# can use this command to view it, quickly check for errors

We can hold these data for future use by writing the subset back to disk.

In [ ]:
with open('../data/spatial/chicago_zips_2017.geojson', 'w') as outfile:
    outfile.write(chicago_json)

## Data Inspection

Now that we have our GeoJSON string for Chicago, let's visually inspect our boundaries to see if they make sense.  We can use a quick and dirty polygon plot.

In [ ]:
# GeoJSONDataSource takes a string, which we just made; IDs Lat/Long, build patches based on them then fill with blue

# Define GeoJSON source
chi_src = bm.GeoJSONDataSource(geojson=chicago_json)

# Define location of output file
bp.output_file('../figs/chicago_access.html')

# Plot blue polys with white lines
p = bp.figure(height=900, width=900, lod_threshold=1)
p.patches(xs='xs', ys='ys', fill_color='blue', source=chi_src)
p.multi_line(xs='xs', ys='ys', line_color='white', line_width=0.3, source=chi_src)

# Show the figure
bp.save(p)

show_iframe('../figs/chicago_access.html', iwidth=950, iheight=850)

# Projection might be off--this is OK if you are just doing exploratory mapping, but need to fix it up if you are presenting the data

## Joining Access Data to Spatial Data

One of the most useful projects I have come across for spatial analysis has been [GeoPandas](http://geopandas.org/). It's kind of exactly what it sounds like. This project combines spatial data operations with the flexible pandas DataFrame. Among other things, it makes it much more convenient to join flat data to polygons, so we will use it to assign access values (and associated colors) to each zip code. 

First, let's create a consolidated GeoDataFrame.

In [ ]:
import matplotlib as mpl

In [ ]:
# Read in chicago zips into a GeoDataFrame
chi_df = gpd.read_file('../data/spatial/chicago_zips_2017.geojson').rename(columns = {'ZCTA5CE10': 'zip_code'})

# Set the projection to be consistent with Bokeh tiles (Mercator)
chi_df = chi_df.to_crs({'init': 'epsg:3857'})

# Set zip code to index to facilitate joining
for df in [access, chi_df]:
    if 'zip_code' in df.columns:
        # Make sure zip codes are integers in both sets
        df['zip_code'] = df['zip_code'].astype(int)
        df.set_index('zip_code', inplace=True)
        
# Join the access data to the spatial data
for c in access.columns:
    chi_df[c] = access[c].loc[chi_df.index]


Second, we need to choose colors for the different values we observe. We want color intensity to have consistent meaning across each group.  In other words, long distances for grocery store purchases should map (at least in intensity) to long distances in restaurants. Note that this is an arguable point, because it imposes restrictions on us in a sense. If we map intensity across different variables, we should probably use the same color to reinforce the point. In this particular instance, we will partially violate this idea by using one color to map product types (e.g grocery stores and restaurants), and another to map income groups (e.g. low- and high-income).

In [ ]:
# Capture minimum and maximum values within each driver bucket
overall_min = access['overall'].min()
overall_max = access['overall'].max()
prod_type_min = access[['grocery_stores', 'restaurants']].min().min()
prod_type_max = access[['grocery_stores', 'restaurants']].max().max()
inc_grp_min = access[['low_income', 'high_income']].min().min()
inc_grp_max = access[['low_income', 'high_income']].max().max()

# Capture normalizer objects (they map data to the [0,1] interval)
norm = {
    'overall': mpl.colors.Normalize(vmin=overall_min, vmax=overall_max),
    'prod_type': mpl.colors.Normalize(vmin=prod_type_min, vmax=prod_type_max),
    'inc_grp': mpl.colors.Normalize(vmin=inc_grp_min, vmax=inc_grp_max)
}

# Generate color mappers; ScalarMappable=linear
cmapper = {
    'overall': cmap.ScalarMappable(norm=norm['overall'], cmap='PiYG'),
    'prod_type': cmap.ScalarMappable(norm=norm['prod_type'], cmap='RdBu'),
    'inc_grp': cmap.ScalarMappable(norm=norm['inc_grp'], cmap='PuOr')
}

# Map colors to values
chi_df['overall_color'] = chi_df['overall'].apply(lambda x: mpl.colors.to_hex(cmapper['overall'].to_rgba(x)))
for c in ['grocery_stores', 'restaurants']:
    chi_df[c + '_color'] = chi_df[c].apply(lambda x: mpl.colors.to_hex(cmapper['prod_type'].to_rgba(x)))
for c in ['low_income', 'high_income']:
    chi_df[c + '_color'] = chi_df[c].apply(lambda x: mpl.colors.to_hex(cmapper['inc_grp'].to_rgba(x)))

## A Basic Choropleth

Now we are in a position to get some preliminary insight.  For the time being, we will stick with evaluating overall merchant distances, until we get the functionality we need.  Since we have several variables, we will seek to build a function that takes a variable as an argument.

In [34]:
# Define data source
chi_access_src = bm.GeoJSONDataSource(geojson=chi_df.reset_index().to_json())

def plot_dist(v, height_in=900, width_in=750):
    # Define location of output file
    outf = '../figs/chicago_access_{}.html'.format(v)
    bp.output_file(outf)
    
    # Define figure
    fig = bp.figure(title='Median Merchant Distance: {}'.format(v.title()),
                       height=height_in, width=width_in)
    
    # Add zip polygons
    fig.patches(xs='xs', ys='ys', alpha=0.9, source=chi_access_src,
                fill_color={'field': v+'_color'}, line_width=1, line_color='#626770')
    
    # Define hover appearance (in raw HTML)
    hover_html = '''
    <div><b>ZIP Code</b>: @zip_code</div>
    <div><b>Merchant Distance</b>: @{}</div>
    '''.format(v)
    hover = bm.HoverTool(
        point_policy='follow_mouse',
        tooltips=(hover_html)
    )
    
    # Define tap tool
    tap = bm.TapTool()
    fig.add_tools(hover, tap)
    
    # Add map tile
    fig.add_tile(CARTODBPOSITRON_RETINA)
    
    # Get rid of axis lines
    fig.xaxis.visible = False
    fig.yaxis.visible = False
    fig.grid.visible = False
    
    # Show the figure
    bp.save(fig)
    return outf

ovr = plot_dist('overall')
show_iframe(ovr, iwidth=850, iheight=950)

<iframe src=../figs/chicago_access_overall.html width=850 height=950></iframe>


Now we are cooking with gas.

In [ ]:
import geojsonio as gjio

In [ ]:
gjio.display(chicago_json)